In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import os 
import sys
import sqlite3
import pickle
from ast import literal_eval
from collections import Counter

In [2]:
%load_ext autoreload
%autoreload 2
sys.path.append(os.path.abspath('../src'))
import gather_data as data_agg

/home/owner/Documents/Github_projects/Netflix_TMDB/venv1/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


## Set up 

In [3]:
db_location = '../Data/netflix_viewing1.db'
netflix_path = '' # Don't actually use this 
user_id = 'malcolm'

with open('../Data/api_key.pkl', 'rb') as hnd:
    key = pickle.load(hnd)['api_key']

In [4]:
con = sqlite3.connect(db_location)
cursor = con.cursor()

In [5]:
cursor.execute('SELECT name FROM sqlite_master WHERE type=\'table\' ORDER BY name').fetchall()

[('Movies',), ('TV_Shows',), ('User_Shows',)]

In [6]:
cursor.execute('pragma table_info(user_shows)').fetchall()

[(0, 'Title', 'TEXT', 0, None, 0),
 (1, 'Date Watched', 'TIMESTAMP', 0, None, 0),
 (2, 'Show Name', 'TEXT', 0, None, 0),
 (3, 'Season', 'TEXT', 0, None, 0),
 (4, 'Episode Name', 'TEXT', 0, None, 0),
 (5, 'TV Show flag', 'TEXT', 0, None, 0),
 (6, 'User_ID', 'TEXT', 0, None, 0),
 (7, 'Date Logged', 'TEXT', 0, None, 0)]

In [7]:
cursor.execute('pragma table_info(Movies)').fetchall()

[(0, 'Input Movie Title', 'TEXT', 0, None, 0),
 (1, 'original_title', 'TEXT', 0, None, 0),
 (2, 'budget', 'REAL', 0, None, 0),
 (3, 'homepage', 'TEXT', 0, None, 0),
 (4, 'imdb_id', 'TEXT', 0, None, 0),
 (5, 'overview', 'TEXT', 0, None, 0),
 (6, 'popularity', 'REAL', 0, None, 0),
 (7, 'release_date', 'TEXT', 0, None, 0),
 (8, 'revenue', 'REAL', 0, None, 0),
 (9, 'runtime', 'REAL', 0, None, 0),
 (10, 'vote_average', 'REAL', 0, None, 0),
 (11, 'vote_count', 'REAL', 0, None, 0),
 (12, 'tagline', 'TEXT', 0, None, 0),
 (13, 'Genres', 'TEXT', 0, None, 0),
 (14, 'Genre IDs', 'TEXT', 0, None, 0),
 (15, 'Number of Search Results', 'INTEGER', 0, None, 0),
 (16, 'Cosine Distance', 'REAL', 0, None, 0),
 (17, 'Date Pulled', 'DATE', 0, None, 0)]

In [8]:
cursor.execute('pragma table_info(TV_Shows)').fetchall()

[(0, 'homepage', 'TEXT', 0, None, 0),
 (1, 'overview', 'TEXT', 0, None, 0),
 (2, 'popularity', 'REAL', 0, None, 0),
 (3, 'vote_average', 'REAL', 0, None, 0),
 (4, 'vote_count', 'INTEGER', 0, None, 0),
 (5, 'name', 'TEXT', 0, None, 0),
 (6, 'id', 'INTEGER', 0, None, 0),
 (7, 'in_production', 'INTEGER', 0, None, 0),
 (8, 'type', 'TEXT', 0, None, 0),
 (9, 'status', 'TEXT', 0, None, 0),
 (10, 'number_of_seasons', 'INTEGER', 0, None, 0),
 (11, 'number_of_episodes', 'INTEGER', 0, None, 0),
 (12, 'n_production_companies', 'INTEGER', 0, None, 0),
 (13, 'primary_production_co', 'TEXT', 0, None, 0),
 (14, 'runtime', 'INTEGER', 0, None, 0),
 (15, 'release_date', 'TEXT', 0, None, 0),
 (16, 'n_network', 'INTEGER', 0, None, 0),
 (17, 'primary_network', 'TEXT', 0, None, 0),
 (18, 'Genres', 'TEXT', 0, None, 0),
 (19, 'Genre IDs', 'TEXT', 0, None, 0),
 (20, 'Number of Search Results', 'INTEGER', 0, None, 0),
 (21, 'Input TV Show Title', 'TEXT', 0, None, 0),
 (22, 'Cosine Distance', 'INTEGER', 0, None, 

## Get TV Data

In [9]:
tv_show_sql = f"""
select * 
from user_shows user
left join TV_Shows tv
on user.`Show Name` = tv.`Input TV Show Title`
where user_id='{user_id}' and `TV Show Flag`='TV Show'
"""

In [10]:
tv_show_df = pd.read_sql(tv_show_sql, con)
tv_show_df['Date Watched'] = pd.to_datetime(tv_show_df['Date Watched'])
tv_show_df['Genres'] = tv_show_df['Genres'].apply(literal_eval)
tv_show_df.head()

,Title,Date Watched,Show Name,Season,Episode Name,TV Show flag,User_ID,Date Logged,homepage,overview,...,runtime,release_date,n_network,primary_network,Genres,Genre IDs,Number of Search Results,Input TV Show Title,Cosine Distance,Date Pulled
0,Dating Around: Season 1: Gurki,2019-03-03,Dating Around,Season 1,Gurki,TV Show,malcolm,2021-02-15 09:13:33.870762,https://www.netflix.com/title/80204889,"In each episode of flirtations and fails, one ...",...,30,2019-02-14,1,Netflix,[Reality],[10764],2,Dating Around,0,2021-02-15
1,Dating Around: Season 1: Luke,2019-03-03,Dating Around,Season 1,Luke,TV Show,malcolm,2021-02-15 09:13:33.870762,https://www.netflix.com/title/80204889,"In each episode of flirtations and fails, one ...",...,30,2019-02-14,1,Netflix,[Reality],[10764],2,Dating Around,0,2021-02-15
2,The Umbrella Academy: Season 1: We Only See Ea...,2019-02-19,The Umbrella Academy,Season 1,We Only See Each Other at Weddings and Funerals,TV Show,malcolm,2021-02-15 09:13:33.870762,https://www.netflix.com/title/80186863,A dysfunctional family of superheroes comes to...,...,55,2019-02-15,1,Netflix,"[Action & Adventure, Sci-Fi & Fantasy, Drama]","[10759, 10765, 18]",2,The Umbrella Academy,0,2021-02-15
3,American Vandal: Season 1: Nailed,2018-11-19,American Vandal,Season 1,Nailed,TV Show,malcolm,2021-02-15 09:13:33.870762,https://www.netflix.com/title/80117545,A true-crime satire that explores the aftermat...,...,35,2017-09-15,1,Netflix,"[Comedy, Crime]","[35, 80]",1,American Vandal,0,2021-02-15
4,American Vandal: Season 1: A Limp Alibi,2018-11-19,American Vandal,Season 1,A Limp Alibi,TV Show,malcolm,2021-02-15 09:13:33.870762,https://www.netflix.com/title/80117545,A true-crime satire that explores the aftermat...,...,35,2017-09-15,1,Netflix,"[Comedy, Crime]","[35, 80]",1,American Vandal,0,2021-02-15


In [11]:
tv_show_summary = tv_show_df.groupby('Show Name').apply(lambda x:data_agg.get_series_watched_gb(x))\
    .sort_values('Total Time Watched (mins)',ascending=False)
tv_show_summary

,Number of Episodes,First Episode Watcheed,Last Episode Watched,Number of Seasons Watched,Total Time Watched (mins),Total Time Watched (hrs),Normal Episode Length,Longest Time to Watch Season,Season took Longest to Watch,# of Episode for Longest to Watch,Longest Days/Episode,Season with Longest Days/Episode,# of Episode for Longest Days/Episode,Shortest Time to Watch Season,Season took Shortest to Watch,# of Episode for Shortest Time to Watch,Shortest Days/Episode,Season with Shortest Days/Episode,# of Episode for Shortest Days/Episode
Show Name,,,,,,,,,,,,,,,,,,,
Friends,171,2016-06-16,2017-09-23,9,4275,71.25,25,10 days,Season 7,24,0 days 10:00:00,Season 7,24,0 days,Season 1,2,0 days 00:00:00,Season 1,2
Dexter,78,2014-06-27,2016-06-12,7,3900,65.00,50,19 days,Season 4,12,1 days 14:00:00,Season 4,12,1 days,Season 3,6,0 days 02:00:00,Season 5,12
White Collar,81,2016-06-26,2016-10-15,6,3402,56.70,42,80 days,Season 2,16,5 days 00:00:00,Season 2,16,1 days,Season 6,6,0 days 03:00:00,Season 4,16
The Blacklist,79,2014-10-03,2018-01-28,4,3397,56.62,43,127 days,Season 4,22,5 days 18:32:43.636363,Season 4,22,16 days,Season 3,23,0 days 16:41:44.347826,Season 3,23
30 Rock,134,2016-11-05,2017-09-23,7,2948,49.13,22,322 days,Season 1,20,16 days 02:24:00,Season 1,20,1 days,Season 7,12,0 days 02:00:00,Season 7,12
House of Cards,58,2013-10-04,2017-06-04,5,2900,48.33,50,23 days,Season 2,13,1 days 18:27:41.538461,Season 2,13,1 days,Season 5,8,0 days 03:00:00,Season 5,8
The Office (U.S.),115,2016-06-11,2018-09-29,7,2530,42.17,22,764 days,Season 6,20,38 days 04:48:00,Season 6,20,0 days,Season 7,2,0 days 00:00:00,Season 7,2
Parks and Recreation,90,2016-09-06,2018-07-07,6,1980,33.00,22,441 days,Season 4,22,20 days 01:05:27.272727,Season 4,22,0 days,Season 1,1,0 days 00:00:00,Season 1,1
Orange Is the New Black,29,2013-10-12,2016-06-21,3,1740,29.00,60,706 days,Season 2,3,235 days 08:00:00,Season 2,3,4 days,Season 4,13,0 days 07:23:04.615384,Season 4,13


In [12]:
tv_show_df[['Date Watched', 'Genres']].head()

,Date Watched,Genres
0,2019-03-03,[Reality]
1,2019-03-03,[Reality]
2,2019-02-19,"[Action & Adventure, Sci-Fi & Fantasy, Drama]"
3,2018-11-19,"[Comedy, Crime]"
4,2018-11-19,"[Comedy, Crime]"


In [13]:
tv_show_df['Date Watched'].describe()

count                    1357
unique                    374
top       2017-05-14 00:00:00
freq                       27
first     2013-09-14 00:00:00
last      2019-03-03 00:00:00
Name: Date Watched, dtype: object

In [14]:
def monthly_agg(df):
    
    out = {}
    # Genres
    all_genres = []
    for x in df['Genres'].tolist():
        all_genres.extend(x)
    genre_cnts = Counter(all_genres).most_common()
    if len(genre_cnts) == 0:
        out['Most Watched Genre'] = None
        out['Most Watched Genre Count'] = None
        out['Most Watched Time (mins)'] = None
        out['Second Genre'] = None
        out['Second Genre Count'] = None
    
    elif len(genre_cnts) == 1:
        out['Most Watched Genre'] = genre_cnts[0][0]
        out['Most Watched Genre Count'] = genre_cnts[0][1]
        out['Most Watched Time (mins)'] = np.sum(np.where(df['Genres']\
                                                          .apply(lambda x: True if 'Drama' in x else False)
                                                          , df['runtime'], 0))
        out['Second Genre'] = None
        out['Second Genre Count'] = None
    else:
        out['Most Watched Genre'] = genre_cnts[0][0]
        out['Most Watched Genre Count'] = genre_cnts[0][1]
        out['Most Watched Time (mins)'] = np.sum(np.where(df['Genres']\
                                                          .apply(lambda x: True if 'Drama' in x else False)
                                                          , df['runtime'], 0))
        out['Second Genre'] = genre_cnts[1][0]
        out['Second Genre Count'] = genre_cnts[1][1]
        
    
    # Other metrics
    out['Minutes Watched'] = df['runtime'].sum()
    out['Hours Watched'] = np.round(df['runtime'].sum()/60, 2)
    
    out_series = pd.Series(out)
    return(out_series)

In [15]:
# tv_show_sm = tv_show_df[tv_show_df['Date Watched'].between('2018-01-01', '2019-01-01')]
tv_show_sm = tv_show_df.copy()
tv_show_sm.set_index('Date Watched', inplace=True)
tv_show_sm.shape

(1357, 31)

In [16]:
tv_show_sm

,Title,Show Name,Season,Episode Name,TV Show flag,User_ID,Date Logged,homepage,overview,popularity,...,runtime,release_date,n_network,primary_network,Genres,Genre IDs,Number of Search Results,Input TV Show Title,Cosine Distance,Date Pulled
Date Watched,,,,,,,,,,,,,,,,,,,,,
2019-03-03,Dating Around: Season 1: Gurki,Dating Around,Season 1,Gurki,TV Show,malcolm,2021-02-15 09:13:33.870762,https://www.netflix.com/title/80204889,"In each episode of flirtations and fails, one ...",4.056,...,30,2019-02-14,1,Netflix,[Reality],[10764],2,Dating Around,0,2021-02-15
2019-03-03,Dating Around: Season 1: Luke,Dating Around,Season 1,Luke,TV Show,malcolm,2021-02-15 09:13:33.870762,https://www.netflix.com/title/80204889,"In each episode of flirtations and fails, one ...",4.056,...,30,2019-02-14,1,Netflix,[Reality],[10764],2,Dating Around,0,2021-02-15
2019-02-19,The Umbrella Academy: Season 1: We Only See Ea...,The Umbrella Academy,Season 1,We Only See Each Other at Weddings and Funerals,TV Show,malcolm,2021-02-15 09:13:33.870762,https://www.netflix.com/title/80186863,A dysfunctional family of superheroes comes to...,354.616,...,55,2019-02-15,1,Netflix,"[Action & Adventure, Sci-Fi & Fantasy, Drama]","[10759, 10765, 18]",2,The Umbrella Academy,0,2021-02-15
2018-11-19,American Vandal: Season 1: Nailed,American Vandal,Season 1,Nailed,TV Show,malcolm,2021-02-15 09:13:33.870762,https://www.netflix.com/title/80117545,A true-crime satire that explores the aftermat...,17.015,...,35,2017-09-15,1,Netflix,"[Comedy, Crime]","[35, 80]",1,American Vandal,0,2021-02-15
2018-11-19,American Vandal: Season 1: A Limp Alibi,American Vandal,Season 1,A Limp Alibi,TV Show,malcolm,2021-02-15 09:13:33.870762,https://www.netflix.com/title/80117545,A true-crime satire that explores the aftermat...,17.015,...,35,2017-09-15,1,Netflix,"[Comedy, Crime]","[35, 80]",1,American Vandal,0,2021-02-15
2018-11-18,American Vandal: Season 1: Hard Facts: Vandali...,American Vandal,Season 1,Hard Facts: Vandalism and Vulgarity,TV Show,malcolm,2021-02-15 09:13:33.870762,https://www.netflix.com/title/80117545,A true-crime satire that explores the aftermat...,17.015,...,35,2017-09-15,1,Netflix,"[Comedy, Crime]","[35, 80]",1,American Vandal,0,2021-02-15
2018-11-18,American Vandal: Season 2: The Brownout,American Vandal,Season 2,The Brownout,TV Show,malcolm,2021-02-15 09:13:33.870762,https://www.netflix.com/title/80117545,A true-crime satire that explores the aftermat...,17.015,...,35,2017-09-15,1,Netflix,"[Comedy, Crime]","[35, 80]",1,American Vandal,0,2021-02-15
2018-09-29,The Office (U.S.): Season 6: Niagara: Part 1,The Office (U.S.),Season 6,Niagara: Part 1,TV Show,malcolm,2021-02-15 09:13:33.870762,http://www.nbc.com/The_Office/,The everyday lives of office employees in the ...,110.285,...,22,2005-03-24,1,NBC,[Comedy],[35],1,The Office (U.S.),0,2021-02-15
2018-07-07,"The Office (U.S.): Season 7: Goodbye, Michael",The Office (U.S.),Season 7,"Goodbye, Michael",TV Show,malcolm,2021-02-15 09:13:33.870762,http://www.nbc.com/The_Office/,The everyday lives of office employees in the ...,110.285,...,22,2005-03-24,1,NBC,[Comedy],[35],1,The Office (U.S.),0,2021-02-15


In [17]:
tv_show_sm.columns

Index(['Title', 'Show Name', 'Season', 'Episode Name', 'TV Show flag',
       'User_ID', 'Date Logged', 'homepage', 'overview', 'popularity',
       'vote_average', 'vote_count', 'name', 'id', 'in_production', 'type',
       'status', 'number_of_seasons', 'number_of_episodes',
       'n_production_companies', 'primary_production_co', 'runtime',
       'release_date', 'n_network', 'primary_network', 'Genres', 'Genre IDs',
       'Number of Search Results', 'Input TV Show Title', 'Cosine Distance',
       'Date Pulled'],
      dtype='object')

In [18]:
monthly_gb = tv_show_sm.groupby(pd.Grouper(freq='M')).apply(lambda x:monthly_agg(x))
monthly_gb = monthly_gb[monthly_gb['Most Watched Genre'].notnull()]
monthly_gb.head(10)

,Most Watched Genre,Most Watched Genre Count,Most Watched Time (mins),Second Genre,Second Genre Count,Minutes Watched,Hours Watched
Date Watched,,,,,,,
2013-09-30,Comedy,5.0,72.0,Drama,3.0,116.0,1.93
2013-10-31,Drama,26.0,1430.0,Comedy,16.0,1482.0,24.70
2014-03-31,Comedy,1.0,0.0,Action & Adventure,1.0,22.0,0.37
2014-04-30,Drama,7.0,350.0,None,NaN,350.0,5.83
2014-05-31,Drama,19.0,613.0,Comedy,15.0,657.0,10.95
2014-06-30,Crime,8.0,352.0,Drama,8.0,396.0,6.60
2014-07-31,Drama,19.0,846.0,Crime,13.0,846.0,14.10
2014-10-31,Drama,1.0,43.0,Crime,1.0,43.0,0.72
2014-11-30,Drama,11.0,473.0,Crime,11.0,473.0,7.88


In [19]:
all_genres = []
lengths = []
for x in tv_show_df['Genres'].tolist():
    all_genres.extend(x)
    lengths.append(len(x))

In [20]:
tv_show_sm['Show Name'].value_counts()

Friends                              171
30 Rock                              134
The Office (U.S.)                    115
Parks and Recreation                  90
White Collar                          81
The Blacklist                         79
Dexter                                78
House of Cards                        58
Unbreakable Kimmy Schmidt             39
Brickleberry                          36
BoJack Horseman                       36
Orange Is the New Black               29
Marvel's Daredevil                    26
Narcos                                21
Black Mirror                          19
Peaky Blinders                        18
Scrubs                                18
How to Get Away With Murder           17
Gossip Girl                           16
Chelsea                               14
Marvel's Luke Cage                    13
Archer                                13
Arrow                                 12
That '70s Show                        11
Queen of the Sou

In [21]:
genre_cnts = Counter(all_genres).most_common()
genre_cnts

[('Drama', 829),
 ('Comedy', 813),
 ('Crime', 386),
 ('Mystery', 218),
 ('Animation', 107),
 ('Action & Adventure', 99),
 ('Sci-Fi & Fantasy', 69),
 ('Talk', 14),
 ('Family', 13),
 ('Documentary', 5),
 ('Reality', 2)]

In [22]:
genre_cnts[0][1]

829

In [23]:
Counter(lengths).most_common()

[(1, 535), (2, 476), (3, 316), (4, 30)]

## Graphs

In [68]:
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash.dependencies as dd
import plotly.graph_objs as go

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


In [69]:
monthly_plt_df = monthly_gb.reset_index()
monthly_plt_df['Most Watched Genre'] = "Most Watched Genre: " + monthly_plt_df['Most Watched Genre']
monthly_plt_df['Date Watched'] = pd.to_datetime(monthly_plt_df['Date Watched'])
monthly_plt_df.head()

,Date Watched,Most Watched Genre,Most Watched Genre Count,Most Watched Time (mins),Second Genre,Second Genre Count,Minutes Watched,Hours Watched
0,2013-09-15,Most Watched Genre: Comedy,2.0,0.0,None,NaN,44.0,0.73
1,2013-09-22,Most Watched Genre: Comedy,3.0,72.0,Drama,3.0,72.0,1.20
2,2013-10-06,Most Watched Genre: Drama,10.0,500.0,None,NaN,500.0,8.33
3,2013-10-13,Most Watched Genre: Drama,7.0,390.0,Comedy,4.0,390.0,6.50
4,2013-10-20,Most Watched Genre: Comedy,10.0,540.0,Drama,9.0,562.0,9.37


In [72]:
px.line(monthly_plt_df, 'Date Watched', 'Hours Watched'
       , hover_name='Most Watched Genre')

In [73]:
montlhy_line = go.Line(x=monthly_plt_df['Date Watched']
                       , y=monthly_plt_df['Hours Watched'])
layout = dict(title='Monthly Hours Watched'
             , xaxis = dict(title='Date')
             , yaxis = dict(title='Sum of Usage Metric'))
fig = go.Figure(data = montlhy_line, layout=layout)
fig.show()

/home/owner/anaconda3/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




## Graveyard

In [ ]:
user_df = pd.read_sql("select * from user_shows where user_id='malcolm'", con)
user_df.head()

In [ ]:
tv_df = pd.read_sql("select * from TV_Shows", con)
tv_df.head()

In [ ]:
user_df['TV Show flag'].value_counts()

In [ ]:
user_df.head()